# ETL

### Importación de Librerías

In [32]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Lectura de los Archivos:

A la lectura se le agregra "na_values=['SD', 'sd']", porque este parámetro indica qué valores se deben considerar como valores nulos (NaN) durante la lectura de datos

In [33]:
hechos = pd.read_excel('datasets/homicidios.xlsx', sheet_name='HECHOS', na_values=['SD', 'sd', 'SD-SD'])
victimas = pd.read_excel('datasets/homicidios.xlsx', sheet_name='VICTIMAS', na_values=['SD', 'sd', 'SD-SD'])

-------------------------------------------------------------------------------------------------------

## Transformación:

### Hechos

In [34]:
hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


#### Eliminacion de columnas 

In [35]:
hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

La columna FECHA será conservada, mientras que las columnas AAAA, DD y MM, que hacen referencia a información de la fecha, serán eliminadas

In [36]:
hechos.drop(['AAAA', 'MM','DD'], axis=1 , inplace=True)

Para la ubicación del hecho, se ha decidido utilizar las columnas Dirección Normalizada, pos X y pos Y, que hacen referencia a la latitud y la longitud respectivamente. Por lo tanto, se procederá a eliminar las siguiente columnas:
* LUGAR_DEL_HECHO
* Calle 
* Altura
* Cruce
* XY (CABA)

In [37]:
hechos.drop(['Dirección Normalizada','LUGAR_DEL_HECHO', 'Calle', 'Altura' ,'Cruce','XY (CABA)'], axis=1 , inplace=True)

#### Valores nulos

In [38]:
# Ahora identifico la cantidad de valores nulos que qudaron en el dataset
for column in hechos.columns:
    print(f'Hay {hechos[column].isna().sum()} valores nulos en {column}')

Hay 0 valores nulos en ID
Hay 0 valores nulos en N_VICTIMAS
Hay 0 valores nulos en FECHA
Hay 1 valores nulos en HORA
Hay 1 valores nulos en HH
Hay 0 valores nulos en TIPO_DE_CALLE
Hay 0 valores nulos en COMUNA
Hay 0 valores nulos en pos x
Hay 0 valores nulos en pos y
Hay 5 valores nulos en PARTICIPANTES
Hay 9 valores nulos en VICTIMA
Hay 23 valores nulos en ACUSADO


Se identifico irregularidades en la columna de "Comuna", donde la presencia de "Comuna 0" indica una inconsistencia, ya que las comunas deben estar en el rango de 1 a 15. Estos casos también serán tratados como valores nulos. Además, se observaron puntos en la columnas "Pos X" y "Pos Y", y se decidira reemplazarlos por valores nulos para mantener la consistencia en la repretanción de coordenadas geograficas

In [39]:
hechos[hechos['COMUNA'] == 0]

,ID,N_VICTIMAS,FECHA,HORA,HH,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,20:35:00,20.0,CALLE,0,.,.,PEATON-SD,PEATON,NaN
139,2016-0174,1,2016-12-27,00:00:00,0.0,AUTOPISTA,0,.,.,NaN,NaN,NaN


In [40]:
latitud_longitud = hechos[(hechos['pos x'] == '.') & (hechos['pos y'] == '.')]
latitud_longitud.head(5)

,ID,N_VICTIMAS,FECHA,HORA,HH,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,20:00:00,20.0,AUTOPISTA,13,.,.,MOTO-SD,MOTO,NaN
106,2016-0136,1,2016-10-25,00:00:00,0.0,AUTOPISTA,4,.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,20:35:00,20.0,CALLE,0,.,.,PEATON-SD,PEATON,NaN
139,2016-0174,1,2016-12-27,00:00:00,0.0,AUTOPISTA,0,.,.,NaN,NaN,NaN
176,2017-0042,1,2017-04-10,09:00:00,9.0,GRAL PAZ,14,.,.,MOTO-CARGAS,MOTO,CARGAS


In [41]:
# Reemplazo de la columna "Comuna" los valores que son 0 y los convierto en NaN
hechos.loc[hechos['COMUNA'] == 0, 'COMUNA'] = np.nan

# Reemplazo los . por valores nulos en las columnas pos x y pos y
hechos.loc[hechos['pos x'] == '.', 'pos x'] = np.nan
hechos.loc[hechos['pos y'] == '.', 'pos y'] = np.nan

In [42]:
hechos.head(5)

,ID,N_VICTIMAS,FECHA,HORA,HH,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,04:00:00,4.0,AVENIDA,8.0,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,01:15:00,1.0,GRAL PAZ,9.0,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,07:00:00,7.0,AVENIDA,1.0,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,00:00:00,0.0,AVENIDA,8.0,-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,05:20:00,5.0,AVENIDA,1.0,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [43]:
# Ahora identifico nuevamente la cantidad de valores nulos que qudaron en el dataframe
for column in hechos.columns:
    print(f'Hay {hechos[column].isna().sum()} valores nulos en {column}')

Hay 0 valores nulos en ID
Hay 0 valores nulos en N_VICTIMAS
Hay 0 valores nulos en FECHA
Hay 1 valores nulos en HORA
Hay 1 valores nulos en HH
Hay 0 valores nulos en TIPO_DE_CALLE
Hay 2 valores nulos en COMUNA
Hay 12 valores nulos en pos x
Hay 12 valores nulos en pos y
Hay 5 valores nulos en PARTICIPANTES
Hay 9 valores nulos en VICTIMA
Hay 23 valores nulos en ACUSADO


#### Modificacion de las columnas:
* Renombrar columnas
* Creacion de una nueva columna

Convierto el nombre de las columnas a minuscula

In [44]:
# Definir un diccionario con los nuevos nombres de las columnas
nuevos_nombres = {
    'ID': 'id_hecho',
    'N_VICTIMAS': 'n_victimas',
    'FECHA': 'fecha',
    'HORA': 'hora',
    'HH': 'hh',
    'LUGAR_DEL_HECHO': 'lugar_del_hecho',
    'TIPO_DE_CALLE': 'tipo_calle',
    'Calle': 'calle',
    'COMUNA': 'comuna',
    'pos x': 'longitud',
    'pos y': 'latitud',
    'PARTICIPANTES': 'participantes',
    'VICTIMA': 'victima',
    'ACUSADO': 'acusado'
}

# Renombrar las columnas del DataFrame
hechos = hechos.rename(columns=nuevos_nombres)
hechos.columns

Index(['id_hecho', 'n_victimas', 'fecha', 'hora', 'hh', 'tipo_calle', 'comuna',
       'longitud', 'latitud', 'participantes', 'victima', 'acusado'],
      dtype='object')

-----------------------------------------------

### VICTIMAS

In [45]:
victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,NaN
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01 00:00:00


#### Eliminacion de columnas

La columna FECHA será conservada, mientras que las columnas AAAA, DD, MM y FECHA_FALLECIMIENTO, que hacen referencia a información de la fecha, serán eliminadas

In [46]:
victimas.drop(['AAAA', 'MM','DD','FECHA_FALLECIMIENTO'], axis=1 , inplace=True)

#### Valores nulos

In [47]:
# Ahora identifico  la cantidad de valores nulos que qudaron en el dataset
for column in victimas.columns:
    print(f'Hay {victimas[column].isna().sum()} valores nulos en {column}')

Hay 0 valores nulos en ID_hecho
Hay 0 valores nulos en FECHA
Hay 11 valores nulos en ROL
Hay 9 valores nulos en VICTIMA
Hay 6 valores nulos en SEXO
Hay 53 valores nulos en EDAD


#### Modificacion de las columnas:
* Renombrar columnas
* Creacion de una nueva columna

In [48]:
nuevos_nombres_victimas = {
    'ID_hecho': 'id_hecho',
    'FECHA': 'fecha',
    'ROL': 'rol_victima',
    'VICTIMA': 'vehiculo_victima',
    'SEXO': 'sexo_victima',
    'EDAD': 'edad_victima'
}
victimas = victimas.rename(columns= nuevos_nombres_victimas)
victimas.columns

Index(['id_hecho', 'fecha', 'rol_victima', 'vehiculo_victima', 'sexo_victima',
       'edad_victima'],
      dtype='object')

In [49]:
victimas['sexo_victima'].unique()

array(['MASCULINO', 'FEMENINO', nan], dtype=object)

In [50]:
victimas['edad_victima'] = pd.to_numeric(victimas['edad_victima'], errors='coerce')
victimas['edad_victima'].replace(np.nan, round(np.mean(victimas['edad_victima']), 0), inplace=True)

In [51]:
victimas['edad_victima'].unique()

array([19., 70., 30., 18., 29., 22., 16., 59., 65., 34., 41., 50., 38.,
       21., 52., 36., 20., 54., 42., 56., 24., 78., 79., 26., 57., 37.,
       58., 23., 60., 53., 51., 40., 87., 76., 75., 35., 80., 43., 45.,
       67., 27., 55., 49., 81., 25., 33., 46., 83., 39., 28.,  7., 48.,
        4., 82., 32., 17., 47., 61., 10., 95., 73., 84., 66., 85.,  1.,
       15., 13., 77., 44., 31., 62., 74., 71., 11., 86., 69., 72., 12.,
       63., 92., 68., 91., 64.,  5., 88.])

----------------------------

In [22]:
hechos.columns

Index(['id_hecho', 'n_victimas', 'fecha', 'hora', 'hh', 'tipo_calle', 'comuna',
       'longitud', 'latitud', 'participantes', 'victima', 'acusado'],
      dtype='object')

#### Procedo a ver los valores nulos y eliminarlos 

In [23]:
# Ahora identifico  la cantidad de valores nulos que qudaron en el dataset
for column in hechos.columns:
    print(f'Hay {hechos[column].isna().sum()} valores nulos en {column}')

Hay 0 valores nulos en id_hecho
Hay 0 valores nulos en n_victimas
Hay 0 valores nulos en fecha
Hay 1 valores nulos en hora
Hay 1 valores nulos en hh
Hay 0 valores nulos en tipo_calle
Hay 2 valores nulos en comuna
Hay 12 valores nulos en longitud
Hay 12 valores nulos en latitud
Hay 5 valores nulos en participantes
Hay 9 valores nulos en victima
Hay 23 valores nulos en acusado


In [24]:
# Ahora identifico  la cantidad de valores nulos que qudaron en el dataset
for column in victimas.columns:
    print(f'Hay {victimas[column].isna().sum()} valores nulos en {column}')

Hay 0 valores nulos en id_hecho
Hay 0 valores nulos en fecha
Hay 11 valores nulos en rol_victima
Hay 9 valores nulos en vehiculo_victima
Hay 6 valores nulos en sexo_victima
Hay 0 valores nulos en edad_victima


In [25]:
hechos = hechos.dropna()
victimas = victimas.dropna()

In [26]:
# Ahora identifico  la cantidad de valores nulos que qudaron en el dataset
for column in hechos.columns:
    print(f'Hay {hechos[column].isna().sum()} valores nulos en {column}')

Hay 0 valores nulos en id_hecho
Hay 0 valores nulos en n_victimas
Hay 0 valores nulos en fecha
Hay 0 valores nulos en hora
Hay 0 valores nulos en hh
Hay 0 valores nulos en tipo_calle
Hay 0 valores nulos en comuna
Hay 0 valores nulos en longitud
Hay 0 valores nulos en latitud
Hay 0 valores nulos en participantes
Hay 0 valores nulos en victima
Hay 0 valores nulos en acusado


In [27]:
# Ahora identifico  la cantidad de valores nulos que qudaron en el dataset
for column in victimas.columns:
    print(f'Hay {victimas[column].isna().sum()} valores nulos en {column}')

Hay 0 valores nulos en id_hecho
Hay 0 valores nulos en fecha
Hay 0 valores nulos en rol_victima
Hay 0 valores nulos en vehiculo_victima
Hay 0 valores nulos en sexo_victima
Hay 0 valores nulos en edad_victima


## Convertimos los DF a CSV

In [70]:
hechos.to_csv('./datasets/hechos5.csv', index=False)

victimas.to_csv('./datasets/victimas.csv', index=False)